For a easier understanading process, we will go step by step instaed of pipline

# Import package

In [166]:
from pandas import DataFrame as df
from sklearn.preprocessing import minmax_scale
import csv
import numpy as np
import matplotlib.pyplot as plt
proportion = 0.3848

# Load Data

load training,testing,additional training data 

In [500]:
def load_csv(loacation,predicted = True):
    """
    load data from csv file, transform class 0 to label -1 for confidence processing 
    :loacation file path in system
    :predited: check wheather ask for traning data or testing data
    """  
    features = []
    classification = []
    with open(loacation) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            # ignore head  
            if readCSV.line_num == 1:
                continue
            
            features.append(row[1:4609])
            # if ask for training data
            if predicted:
                # transform class 0 to -1    
                if row[4609] == '0':
                    classification.append(-1)
                else:
                    classification.append(1)
    # store features
    features = np.array(features)
    CNN_features = np.array(features[...,:4096])
    GIST_features = np.array(features[...,4096:])
    return CNN_features,GIST_features,classification

def loda_data():
    """
    load training,testing,additional training data respect to CNN, GIST, class 
    """  
  
    CNN_features,GIST_features,classification = load_csv('training.csv')
    t_CNN_features,t_GIST_features, t_classification = load_csv('testing.csv',False)
    ic_CNN_features, ic_GIST_features, ic_classification = load_csv('additional_training.csv')
  
    train = [CNN_features,GIST_features,classification]
    test = [t_CNN_features,t_GIST_features]
    ic_train = [ic_CNN_features, ic_GIST_features, ic_classification]
    return train,test,ic_train

# ic_train means incomplete data, train means complete data, similar as below varibale name
train,test,ic_train = loda_data()

In [598]:
def load_classification():
    # store classfiction information
    classification = train[2].copy()
    ic_classification = ic_train[2].copy()

    #concatenate traning class data and additional traning class data
    # c_xxx mean the combination of traning data and additional traning  data
    c_classification = np.concatenate((classification,ic_classification))
    return classification,ic_classification,c_classification


classification,ic_classification,c_classification = load_classification()    

# the proportion of different class data

In [ ]:
plt.hist(c_classification)

# apply confidence

import confidence to change class label with regrad confidence of all 0 class data as 1

In [518]:
def apply_confidence():
    with open('annotation_confidence.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for i,row in enumerate(readCSV):
            if readCSV.line_num == 1:
                continue
            # complete training data
            if i <= 247:
                if (classification[i-1]) * float(row[1])*100 == -66:
                    classification[i-1] = -100
                else:
                    classification[i-1] = (classification[i-1]) * float(row[1])*100
            # incomplete training data        
            elif (ic_classification[i-248]) * float(row[1])*100 == -66:
                ic_classification[i-248] = -100
            else:
                ic_classification[i-248] = (ic_classification[i-248]) * float(row[1])*100
            return np.concatenate((classification,ic_classification))
        
c_classification = apply_confidence()   

In [ ]:
plt.hist(c_classification)

ignore all class 1 data with confidence 0.66 through specifying the number of each class

In [185]:
# traing data
from imblearn.under_sampling import RandomUnderSampler
strategy = {100:113,66:0,-100:38}
rus = RandomUnderSampler(sampling_strategy=strategy,random_state=0)
new_feas, new_class = rus.fit_resample(train[0].copy(), classification)

In [186]:
# addtional training dataa
strategy = {100:1029,66:0,-100:288}
rus = RandomUnderSampler(sampling_strategy=strategy,random_state=0)
new_icfeas, new_iclass = rus.fit_resample(ic_train[0], ic_classification)

# Imputer (complete missing value)

In [277]:
# concatenate traing data and addtional training dataa
features = np.concatenate((new_feas,new_icfeas))
_class = np.concatenate((new_class,new_iclass))

In [151]:
from sklearn.impute import SimpleImputer

In [278]:
# use most frequent strategy based on obeserving CNN feature
spi = SimpleImputer(strategy = 'most_frequent').fit(features.astype(float))
c_features = spi.transform(features.astype(float))

# feature selection

adopt on L1 sparse solution

In [628]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
"""
:model: feature selection model is saved to apply on testing data
:Selected_ctrain: complete total data after feature selection
"""  
lsvc = LinearSVC(C=0.11,penalty="l1", dual=False).fit((c_features), _class)
model = SelectFromModel(lsvc, prefit=True)
Selected_ctrain = model.transform((c_features))
print('Original feature shape:',c_features.shape)
print('After feature selection, shape:',Selected_ctrain.shape)

Original feature shape: (1468, 4096)
After feature selection, shape: (1468, 692)


# Under sampling ( balanced data)

In [640]:
from imblearn.under_sampling import ClusterCentroids
"""
:X_resampled: resampled training data
:y_resampled: resampled class of training data
"""  
cc = ClusterCentroids(random_state=0)
X_resampled, y_resampled = cc.fit_sample(Selected_ctrain, _class)
print('After under sampling, shape:',X_resampled.shape)

/Users/xingxing/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:968: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


After under sampling, shape: (652, 692)


# MLP model created

In [648]:
from sklearn.neural_network import MLPClassifier
ML = MLPClassifier(alpha = 0.001, hidden_layer_sizes = 110,solver='lbfgs',random_state=1, max_iter=10000,learning_rate = 'adaptive',learning_rate_init = 0.001)

# Tuning hyperparameters

In [32]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'hidden_layer_sizes':[125,120,126,100,127]}
clf = GridSearchCV(ML, parameters)
clf.fit(X_resampled, y_resampled)

In [ ]:
clf.best_estimator_

# inspect the model exprssion ability

In [389]:
def cv(estimator,X_resampled,y_resampled,cv = 5,norm = None):
    """
    cross-validation to improve the classifer performence, 
    print the average accuracy and each split accuracy.
    :y: ground truth
    :pred: predictions classifer made
    """  
    from sklearn.model_selection import cross_validate
    if norm != None:
        X_resampled= minmax_scale(X_resampled)
        X_new = minmax_scale(X_new)
    cv_results = cross_validate(estimator, X_resampled, y_resampled ,cv = cv)
    print(cv_results['test_score'])
    print(sum(cv_results['test_score'])/5)

In [649]:
cv(ML,minmax_scale(X_resampled), y_resampled ,cv = 5)

[0.72519084 0.80916031 0.77692308 0.68461538 0.79230769]
0.7576394597768644


# Save csv results

In [650]:
def save_csv(estimator,x,y,test,directory, func = None,normal = None,):
    """
    fit the model and save the prediction as csv file, alternatively set feature selection
    model and data preprocessing model
    transofrom the label changed by confidence back to normal class label
    :estimator: the model used
    :x: np.array same shape as y:training data
    :y: np.array same shape as x:testing data
    :func: feature selection model
    :normal: data pre-processing model
    :directory: string: file saving path
    """ 
    import pandas as pd
    if func != None:
        test = func.transform(test)
    if normal != None:
        x = normal(x)
    estimator.fit(x,y)
    results = estimator.predict(test)
    predictions = []
    for i, prediction in enumerate(results):
        if prediction > 0:
            predictions.append([i+1,1])
        else:
            predictions.append([i+1,0])
    df = pd.DataFrame(predictions,columns = ['ID','prediction'])
    df.to_csv(directory,index = False)

save_csv(ML,X_resampled,y_resampled,test[0].copy().astype(float),'c0.11 alpha =0.0001 100 results_data.csv',model,minmax_scale)

# Analyze results

In [ ]:
from sklearn.metrics import accuracy_score, hinge_loss, confusion_matrix

In [651]:
def compare_current_best():
    """
    compare the current results to current best results
    print accuracy score between them
    """ 
    pred = []
    with open('c0.11 alpha =0.0001 100 results_data.csv') as csvfile:
      readCSV = csv.reader(csvfile, delimiter=',')
      for i,row in enumerate(readCSV):
        for row in readCSV:
            if readCSV.line_num == 1:
                continue
            pred.append(row[1])
    old_re = []
    with open('c0.11 alpha =0.0001 100 results_data.csv') as csvfile:
      readCSV = csv.reader(csvfile, delimiter=',')
      for i,row in enumerate(readCSV):
        for row in readCSV:
            if readCSV.line_num == 1:
                continue
            old_re.append(row[1])
    print('Classification matched = %d' % accuracy_score(old_re, pred, normalize=False))
    print(confusion_matrix(old_re,pred))     
compare_current_best()

Classification matched = 11465
[[6858  111]
 [ 298 4607]]


In [603]:
def cal_auc(y,pred):
    """
    calculate auc value to inspect model effect
    :y: ground truth
    :pred: predictions classifer made
    """  
    import numpy as np
    from sklearn import metrics
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    print(metrics.auc(fpr, tpr))